In [362]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [363]:
import pandas as pd
import numpy as np
import datetime

In [364]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [365]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_97339/4163294631.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


In [366]:
op_token_columns = ['op_claimed','op_deployed', 'op_from_other_projects','op_to_other_projects','op_to_project']

In [367]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )
group_cols = ['from_name','to_label','to_type','program_map'] + op_token_columns

sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])
                            # sum_distrib_df['from_name'] + ' - ' \
                            #     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(sum_distrib_df[sum_distrib_df['join_key'].str.contains('elodr')])

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_97339/15182629.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()


In [368]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
df = sum_distrib_df.merge(tvl, on='join_key',how='outer')
# display(df[df['join_key'].str.contains('velodr')])

,from_name,program_map,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,join_key,date,include_in_summary,...,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention
4,Alchemix,alETH/WETH: Velodrome,0.0,11500.0,0.0,0.0,0.0,aleth/weth: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Season 1,2023-01-08,NaN,3.352092e+06,3.352092e+06,3.358670e+06,3.358670e+06,NaN,NaN
5,Velodrome,alETH/WETH: Velodrome,0.0,3366.0,0.0,0.0,0.0,aleth/weth: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Season 1,2023-01-08,NaN,3.352092e+06,3.352092e+06,3.358670e+06,3.358670e+06,NaN,NaN
7,Angle,OP/agEUR: Velodrome,0.0,3400.0,0.0,0.0,0.0,op/ageur: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Season 1,2023-02-20,NaN,4.218107e+05,4.218107e+05,4.087959e+05,4.087959e+05,NaN,NaN
8,Velodrome,OP/agEUR: Velodrome,0.0,1014.0,0.0,0.0,0.0,op/ageur: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Season 1,2023-02-20,NaN,4.218107e+05,4.218107e+05,4.087959e+05,4.087959e+05,NaN,NaN
13,Beefy Finance,BIFI/OP: Velodrome,0.0,41000.0,0.0,0.0,0.0,bifi/op: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,4.216617e+05,4.216617e+05,4.925098e+05,4.925098e+05,NaN,NaN
14,Velodrome,BIFI/OP: Velodrome,0.0,20086.0,0.0,0.0,0.0,bifi/op: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,4.216617e+05,4.216617e+05,4.925098e+05,4.925098e+05,NaN,NaN
15,Beefy Finance,BIFI/WETH: Velodrome,0.0,68875.0,0.0,0.0,0.0,bifi/weth: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,1.906687e+06,1.906687e+06,1.942967e+06,1.942967e+06,NaN,NaN
16,Velodrome,BIFI/WETH: Velodrome,0.0,23795.0,0.0,0.0,0.0,bifi/weth: velodrome,2023-03-11,1.0,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,1.906687e+06,1.906687e+06,1.942967e+06,1.942967e+06,NaN,NaN
40,Layer2dao,L2DAO/OP: Velodrome,0.0,14000.0,0.0,0.0,0.0,l2dao/op: velodrome,2022-09-21,1.0,...,Post-Program,Gov Fund - Phase 0,2022-07-20,2022-08-22,4.681078e+05,1.750505e+05,3.585146e+05,1.175015e+05,0.373953,0.327745
41,Velodrome,L2DAO/OP: Velodrome,0.0,6800.0,0.0,0.0,0.0,l2dao/op: velodrome,2022-09-21,1.0,...,Post-Program,Gov Fund - Phase 0,2022-07-20,2022-08-22,4.681078e+05,1.750505e+05,3.585146e+05,1.175015e+05,0.373953,0.327745


In [369]:
# Overrides as needed
def replace_program_names(df, overrides):
    for program, program_override in overrides.items():
        df.loc[df['join_key'] == program, 'from_name'] = program_override
    return df

#Overrides if needed
overrides = {
    'old name'  :   'new name',
}

# Replace program names with overrides
df = replace_program_names(df, overrides)

In [370]:
#Create the aggregate app name field
df['agg_app_name'] = df['app_name'].combine_first(df['from_name'])
df = df.fillna(0) #Fill NA with 0
# display(df)
display(df[df['agg_app_name'].str.contains('elodr')])

,from_name,program_map,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,join_key,date,include_in_summary,...,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention,agg_app_name
91,Velodrome,,4327776.0,4400739.0,47272.0,0.0,0.0,velodrome,2023-03-11,1.0,...,Gov & Partner Fund,2022-07-13,0,2.395393e+08,2.395393e+08,2.395393e+08,2.395393e+08,0.0,0.0,Velodrome


In [371]:
# Now union back again

data_cols = ['agg_app_name','top_level_name', 'program_name',\
               'num_op_override', 'period','op_source', 'start_date','end_date',
               'cumul_net_dollar_flow_at_program_end',
                'cumul_net_dollar_flow',
                'cumul_last_price_net_dollar_flow_at_program_end',
                'cumul_last_price_net_dollar_flow'
                ]
select_cols = data_cols + op_token_columns

group_cols = select_cols[:8] #group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

['agg_app_name', 'top_level_name', 'program_name', 'num_op_override', 'period', 'op_source', 'start_date', 'end_date']


In [372]:
#Get the rank by start_date of each program

sum_distrib_df = sum_distrib_df.reset_index().rename(columns={'index': 'row_num'})

# replace 0s with '9999-12-31'
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == 0, '9999-12-31', sum_distrib_df['start_date'])
# create a new column 'program_rank' based on the 'start_date' column
sum_distrib_df = sum_distrib_df.sort_values(['agg_app_name', 'start_date', 'row_num'])
sum_distrib_df['program_rank'] = sum_distrib_df.groupby('agg_app_name').cumcount() + 1

sum_distrib_df = sum_distrib_df.sort_values(by=['agg_app_name','program_rank'],ascending=[True,True])
# subtract all overridden values fromthe amount I have deployed

# create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
sum_distrib_df['cumulative_num_op_override'] = sum_distrib_df.groupby('agg_app_name')['num_op_override'].cumsum()


# create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
sum_distrib_df['op_deployed_net_override'] = sum_distrib_df['op_deployed'] - sum_distrib_df.groupby('agg_app_name')['cumulative_num_op_override'].shift(1).fillna(0)
# drop the 'cumulative_num_op_override' column
# sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

# replace '9999-12-31' with 0s
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == '9999-12-31', 0, sum_distrib_df['start_date'])

In [373]:
# Override # OP Deployed 
sum_distrib_df['op_deployed'] = sum_distrib_df.apply(lambda x: x['num_op_override'] if x['num_op_override'] > 0 else x['op_deployed'], axis=1)

# replace 0s in 'num_op_override' with the corresponding value in 'op_deployed_net_override'
sum_distrib_df['og_op_deployed'] = sum_distrib_df['op_deployed'] 
sum_distrib_df['op_deployed'] = np.where(sum_distrib_df['num_op_override'] == 0, sum_distrib_df['op_deployed_net_override'], sum_distrib_df['num_op_override'])

#Select all except the last 4 rows
sum_distrib_df = sum_distrib_df.iloc[:, :-4]


In [374]:
# latest tvl metrics
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow'] / sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end']\
            ,np.nan\
)

In [375]:
display(sum_distrib_df)
sum_distrib_df['last_updated'] = pd.to_datetime(datetime.datetime.now())
sum_distrib_df.to_csv('csv_outputs/incentives_stats_summary.csv')
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

,row_num,agg_app_name,top_level_name,program_name,num_op_override,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,net_flows_retention
0,0,1inch,0,0,0.000000e+00,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,25359.0,3.000000e+05,0.0,0.0,0.0,NaN
1,1,Aave,Aave - Liquidity Mining,Aave - Aave - Liquidity Mining,5.000000e+06,Post-Program,Partner Fund,2022-08-04,2022-11-04,3.450056e+08,7.849483e+07,3.419694e+08,7.732963e+07,4820532.0,5.000000e+06,0.0,0.0,0.0,0.226130
2,2,Aelin,Aelin,Aelin,0.000000e+00,Post-Program,Gov Fund - Phase 0,2022-09-12,2022-09-14,-2.827652e+03,-2.778652e+03,-2.826693e+03,-2.777707e+03,880175.0,9.000020e+05,0.0,19827.0,0.0,0.982670
4,4,Alchemix,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,0.000000e+00,During Program,Gov Fund - Season 1,2023-01-08,0,6.704185e+06,6.704185e+06,6.717341e+06,6.717341e+06,0.0,1.486600e+04,0.0,0.0,0.0,NaN
3,3,Alchemix,0,0,0.000000e+00,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21722.0,1.039700e+04,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,Xtoken,Uniswap LM - Phase 1,Xtoken - Uniswap LM - Phase 1,1.666667e+04,Post-Program,Gov Fund - Phase 0,2022-10-26,2022-11-21,2.002074e+06,1.010266e+06,2.137549e+06,1.087476e+06,0.0,1.666667e+04,0.0,0.0,0.0,0.508749
93,93,Xtoken,Uniswap LM - Phase 2,Xtoken - Uniswap LM - Phase 2,2.500000e+04,Post-Program,Gov Fund - Phase 0,2023-01-16,2023-02-08,2.823585e+06,-3.557705e+04,3.027949e+06,7.479739e+04,0.0,2.500000e+04,0.0,0.0,0.0,0.024702
95,95,Zeroex (0x),0,0,0.000000e+00,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,452.0,4.520000e+02,0.0,0.0,0.0,NaN
96,96,Zipswap,0,0,0.000000e+00,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2469.0,2.469000e+03,0.0,0.0,0.0,NaN
